In [6]:
import numpy as np 
import pandas as pd 
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import statsmodels.api as sm
import threading
import pickle
from numba import jit, cuda


In [7]:
train = pd.read_csv("./data/train.csv")
stocks = train['stock_id'].unique()
train = train.sort_values(['date_id','time_id'])
train.head()



,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [8]:
@jit(target_backend='cuda') 
def split(data,var='wap',testing_size=100,pred_size=10):
    X = data.iloc[0:data.shape[0]-pred_size]
    y = data[var].iloc[pred_size:]

    X_train = X.iloc[:X.shape[0]-testing_size]
    X_test = X.iloc[X.shape[0]-testing_size:]

    y_train = y.iloc[:y.shape[0]-testing_size]
    y_test = y.iloc[y.shape[0]-testing_size:].reset_index(drop=True)

    trainTest = {
        'X_train':X_train,
        'X_test':X_test,
        'y_train':y_train,
        'y_test':y_test
    }
    return trainTest

C:\Users\adams\AppData\Local\Temp\ipykernel_4588\72057834.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [9]:
@jit(target_backend='cuda') 
def createForecaster(data,stock,forecasters,var='wap'):
    X_train = data['X_train']
    print("stock: ",stock)
       
    model = ARIMA(X_train[X_train['stock_id'] == stock][var], order=(2,0,0))
        
    forecasters[stock] = model.fit()
    print("created forecaster for stock:",stock)
        
        
        

C:\Users\adams\AppData\Local\Temp\ipykernel_4588\3150368931.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [10]:
@jit(target_backend='cuda') 
def getForecast(data,forecaster,stock,results,var='wap',forecast_dist = 6,method='mean'):
    print("forcasting stock:",stock)
    y_hat = []
    i = 0
    X_test = data['X_test']
    for x in X_test[X_test['stock_id'] == stock][var]:
        forecast = forecaster.forecast(forecast_dist)

        if method == 'mean':
            y_hat.append(np.mean(forecast))
        elif method == 'last':
            y_hat.append(forecast.iloc[-1])
        else:
            print("invalid method")
            break
        
        if i % 10 == 0:
            print(stock,'run:',i)
        forecaster = forecaster.append([x], refit=False)
        
        i += 1

    results[stock] = y_hat
    print("create forecast for stock:",stock)

C:\Users\adams\AppData\Local\Temp\ipykernel_4588\3676412223.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [11]:
@jit(target_backend='cuda') 
def createModel(data,forecasts,stock,models):
    results = {}
    X_test = data['X_test']
    
    y_hat_df = pd.DataFrame(forecasts)
    
    print("creating modl for stock:",stock)
    target = X_test[X_test['stock_id'] == stock]
    target.reset_index(inplace=True)


    ols_model = sm.OLS(list(target['target']),y_hat_df)
    results[stock] = ols_model.fit()
    print(results[stock].summary())

    models[stock] = results
    print("create model for stock:",stock)




    y_hat_df = pd.DataFrame(y_hat)
    
    for stock in stocks:
        print("creating modl for stock:",stock)
        target = X_test[X_test['stock_id'] == stock]
        target.reset_index(inplace=True)


        ols_model = sm.OLS(list(target['target']),y_hat_df)
        results[stock] = ols_model.fit()
        print(results[stock].summary())

    return results

C:\Users\adams\AppData\Local\Temp\ipykernel_4588\1080582510.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [12]:
trainTest = split(train,var='wap',testing_size=500000)


C:\Users\adams\AppData\Local\Temp\ipykernel_4588\72057834.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "split" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\adams\AppData\Local\Temp\ipykernel_4588\72057834.py (1)

File "..\..\..\AppData\Local\Temp\ipykernel_4588\72057834.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "split" was compiled in object mode without forceobj=True.

File "..\..\..\AppData\Local\Temp\ipykernel_4588\72057834.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compila

In [14]:
@jit(target_backend='cuda') 
def runCreateForecast():
    forecasters = {}
    threads = {}
    for stock in stocks:
        threads[stock] = threading.Thread(target=createForecaster,args=(trainTest,stock,forecasters,))

    for stock in stocks:
        threads[stock].start()

    for stock in stocks:
        threads[stock].join()

    return forecasters

    

res = runCreateForecast()

with open('forecasters.pickle', 'wb') as handle:
        pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

C:\Users\adams\AppData\Local\Temp\ipykernel_4588\4047206217.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\adams\AppData\Local\Temp\ipykernel_4588\4047206217.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "runCreateForecast" failed type inference due to: Untyped global name 'trainTest': Cannot determine Numba type of <class 'dict'>

File "..\..\..\AppData\Local\Temp\ipykernel_4588\4047206217.py", line 6:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\adams\AppData\Local\Temp\ipykernel_4588\4047206217.py:1: NumbaWarning: 
Compilation is falling ba

stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: 

c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "createForecaster" was compiled in object mode without forceobj=True.

File "..\..\..\AppData\Local\Temp\ipykernel_4588\3150368931.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is deprecated behaviour that will be removed in Numba 0.59.0.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "..\..\..\AppData\Local\Temp\ipykernel_4588\3150368931.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaDeprecationWarning(msg,


stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: stock: 

C:\Users\adams\AppData\Local\Temp\ipykernel_4588\4047206217.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "runCreateForecast" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\adams\AppData\Local\Temp\ipykernel_4588\4047206217.py (11)

File "..\..\..\AppData\Local\Temp\ipykernel_4588\4047206217.py", line 11:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')


...

c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock:created forecaster for stock: created forecaster for stock:created forecaster for stock:created forecaster for stock: created forecaster for stock:                                             

c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                 created forecaster for stock:      created forecaster for stock: created forecaster for stock:  created forecaster for stock:                                                         1    6210 1621740814051577453197169188815113146411181088955331781731261128172023125261752411013325142106177971842767736115915842127187183176802166138245122109541989518013019098181631611211001191791951054172156136319316711461149601043615821434168139591601541557816685128 1861418692144961523956941111297717183501856911664 203748681175  18230163441582919414338749931745715316484461481471218816132711171345119222189701651505235196113651039913532874391911371452831102124191074717076
90159120
123




















































































































101












79131




199
































































In [ ]:


@jit(target_backend='cuda') 
def runForecast(forecasters):
    

    forecasts = {}
    threads = {}
    for stock in stocks:
        threads[stock] = threading.Thread(target=getForecast,args=(trainTest,forecasters[stock],stock,forecasts,))

    for stock in stocks:
        threads[stock].start()

    for stock in stocks:
        threads[stock].join()

    pd.DataFrame(forecasts).head()

    

with open('forecasters.pickle', 'rb') as handle:
        forecasters = pickle.load(handle)  

res = runForecast(forecasters)

with open('forecasts.pickle', 'wb') as handle:
        pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
@jit(target_backend='cuda') 
def runCreateModel(forecasts):
    

    models = {}
    threads = {}
    for stock in stocks:
        threads[stock] = threading.Thread(target=createModel,args=(trainTest,forecasts,stock,models,))

    for stock in stocks:
        threads[stock].start()

    for stock in stocks:
        threads[stock].join()
  

with open('forecasts', 'rb') as handle:
        forecasts = pickle.load(handle)

models = runCreateModel(forecasts)

with open('linear_models', 'wb') as handle:
        pickle.dump(models, handle, protocol=pickle.HIGHEST_PROTOCOL)